In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient, AWSIoTMQTTClient

# Connection Setting
SHADOW_CLIENT = "ShadowClient1_A0195017E"
HOST_NAME = "*********************.amazonaws.com"
ROOT_CA = "AmazonRootCA1.pem.txt"
PRIVATE_KEY = "****************-private.pem.key"
CERT_FILE = "***************-certificate.pem.crt"
SHADOW_HANDLER = "thing1_A0195017E"

# Automatically called whenever the shadow is updated.
def myShadowUpdateCallback(payload, responseStatus, token):
    print('UPDATE: $aws/things/' + SHADOW_HANDLER + '/shadow/update/#')
    print("payload = " + payload)
    print("responseStatus:" + responseStatus)
    print("token = " + token)

# Create, configure, and connect a shadow client.
myShadowClient = AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME, 8883)
myShadowClient.configureCredentials(ROOT_CA, PRIVATE_KEY,CERT_FILE)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(5)
myShadowClient.connect()

# Create a programmatic representation of the shadow.
myDeviceShadow = myShadowClient.createShadowHandlerWithName(SHADOW_HANDLER, True)

In [ ]:
#process data
import pandas as pd
import time, datetime
import json

#publish data
MVP = pd.read_csv('annual-motor-vehicle-population-by-vehicle-type.csv')
for i in range(len(MVP)):
    msg=MVP.loc[[i]]
    msg.index=["reported"]
    msg=msg.to_json(orient='index')
    msg='{"state":'+msg+'}'
    myDeviceShadow.shadowUpdate(msg,myShadowUpdateCallback, 5)
    time.sleep(0.5)